# Copy instead of modifying to preserve attack parameters

In [1]:
AGENT_NAME = r"Karla\Victims\3-26-5_PPOc_citylearn_challenge_2022_phase_2_('Building_6',)_gSDE_norm_space_SolarPenaltyReward_deep_net_256_40000.zip"
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SURROGATE_PATH = r'Karla\3-26-5 PPOc Karla results\proxies\imitator best.pth'
#RUNS = 5
TRIAL = 'bifurcation DL loss'
SAVE_DIR = 'Karla\3-26-5 PPOc Karla results\imitator (best) fgm eps vs asr' + '/'

In [2]:
from stable_baselines3 import PPO #SAC

from citylearn.data import DataSet

from art.estimators.classification import PyTorchClassifier as classifier
from art.attacks.evasion import FastGradientMethod as FGM

import pandas as pd
import numpy as np
#import os
import torch
#from torch.nn import CrossEntropyLoss

import KBMproject.utilities as utils

from joblib import Parallel, delayed

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
testilons = np.arange(0.02, 0.21, 0.01)
RUNS = len(testilons)

Define RL agent

In [5]:
agents = []
for _ in range (RUNS):
    agents.append(PPO.load(AGENT_NAME))

Create environments

In [6]:
envs = []
for _ in range (RUNS):
    envs.append(utils.make_continuous_env(schema=schema,  
                                seed=42))

In [7]:
cols = utils.make_continuous_env(schema=schema,  
                                seed=42).observation_names

In [8]:
imitator = torch.load(SURROGATE_PATH)

In [9]:
observation_masks = np.ones(agents[0].observation_space.shape)
observation_masks[0:6] = 0 #mask time features
print('masked features:')
cols[0][0:6]

masked features:


['month_cos',
 'month_sin',
 'day_type_cos',
 'day_type_sin',
 'hour_cos',
 'hour_sin']

In [10]:
kwargs = dict(norm=np.inf,
        targeted=True, #True for toggled attack
        )
attacks = []
for run in range(RUNS):
    attacks.append(FGM(estimator=classifier(
                                        model=utils.RegressorLinearWrapper(imitator),
                                        #loss=CrossEntropyLoss(),
                                        loss=utils.CWLoss(),
                                        nb_classes=2, 
                                        input_shape=agents[run].observation_space.shape,
                                        device_type='gpu',
                                        clip_values = (agents[run].observation_space.low.min(),
                                                       agents[run].observation_space.high.max()),
                                        ),
                        eps=testilons[run],
                        **kwargs))

In [11]:
%%time
%%capture
results = Parallel(n_jobs=RUNS, verbose=10, prefer='threads')(delayed(
    utils.eval_bifurcation_continuous_attack)(agent, env, attack) for agent, env, attack in zip(agents, envs, attacks)) 


CPU times: total: 4min 15s
Wall time: 15min 36s


Results is a list of tupples for each run, of the format(KPIs, observations, perturbed observations, epsilons)

In [12]:
kpis = [results[i][0] for i in range(len(results))]
df_kpis = pd.concat(kpis, axis='columns',keys=testilons)

In [13]:
#df_kpis[['mean', 'std', 'variance']] = df_kpis.agg(['mean','std', 'var'], axis='columns')

In [14]:
df_kpis

,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20
cost_function,,,,,,,,,,,,,,,,,,,
annual_peak_average,1.000000,1.000000,1.000000,0.997215,0.999757,1.050238,1.109659,1.260678,1.352241,1.462686,1.542433,1.600686,1.614529,1.586269,1.561921,1.764372,1.693401,1.370627,1.742406
carbon_emissions_total,0.882038,0.902290,0.931238,0.965014,0.999547,1.038416,1.078726,1.121718,1.165965,1.204268,1.229361,1.251298,1.257126,1.264448,1.276643,1.273948,1.270004,1.254261,1.248684
cost_total,0.810960,0.825170,0.851284,0.882379,0.911959,0.947048,0.985648,1.029785,1.073293,1.114691,1.141782,1.169678,1.182209,1.194961,1.209810,1.212965,1.211386,1.202990,1.200668
daily_one_minus_load_factor_average,1.061536,1.029208,1.003364,0.980066,0.964076,0.956914,0.958188,0.954960,0.955599,0.958727,0.959370,0.965913,0.969696,0.964642,0.967617,0.964893,0.962587,0.962683,0.966446
daily_peak_average,0.903235,0.938765,0.977427,1.053270,1.140195,1.220307,1.329314,1.484333,1.572980,1.673540,1.724495,1.766011,1.803628,1.813571,1.866762,1.880800,1.862136,1.846016,1.842676
electricity_consumption_total,0.897960,0.918668,0.947190,0.980092,1.014572,1.054111,1.095386,1.141434,1.188514,1.228142,1.256545,1.280309,1.284555,1.293222,1.304399,1.303809,1.293198,1.279800,1.275349
monthly_one_minus_load_factor_average,0.977648,0.975253,0.974548,0.979579,0.990508,0.993186,0.989386,1.004483,1.007434,1.010288,1.011452,1.016434,1.011499,1.015744,1.014686,1.016583,1.015634,1.017332,1.019428
ramping_average,1.220700,1.352045,1.511753,1.698680,1.887306,2.093086,2.348936,2.592592,2.843029,3.081927,3.245274,3.380171,3.394743,3.459415,3.531228,3.561658,3.501871,3.384042,3.349783
zero_net_energy,1.090012,1.095016,1.099767,1.104293,1.112879,1.120840,1.130864,1.138868,1.149627,1.161160,1.166742,1.170355,1.170070,1.170837,1.172102,1.173172,1.170801,1.164278,1.162565


In [15]:
df_kpis.to_csv(SAVE_DIR + f'{RUNS} run KPIs {TRIAL}.csv')

FGSM using the same parameters in a whitebox attack has an ASR of 0.7.

Increasing the number of restarts decreases the ASR, likely because of differences between the surrogate and vicitm models

In [16]:
MAEs = [results[i][5] for i in range(len(results))]
#print(f'For 5 runs of the random attack \nthe mean ASR is: {np.mean(ASRs):.3f}\nthe STD is: {np.std(ASRs):.3f}')

In [17]:
MAEs

[0.04534847837548398,
 0.06773820070319965,
 0.08850850168803141,
 0.10780556067672953,
 0.12704726447320336,
 0.14594184929056928,
 0.1635075048886419,
 0.18232714059432084,
 0.20163446731839038,
 0.22093515971341435,
 0.23880122292870623,
 0.25701897922328704,
 0.2741419063377685,
 0.2895434977107566,
 0.3048166364809057,
 0.3196210664951008,
 0.33210452756512615,
 0.34628245902396265,
 0.35668057169608297]

In [18]:
pd.DataFrame({'MAEs':MAEs}, index=testilons,).to_csv(SAVE_DIR + f'MAE for varied epsilons {TRIAL}.csv')

In [19]:
for run in range(RUNS):
        pd.DataFrame(results[run][1],columns=cols,).to_csv(SAVE_DIR+f'run {run} obs {TRIAL}.csv',)
        pd.DataFrame(results[run][2],columns=cols,).to_csv(SAVE_DIR+f'run {run} adv obs {TRIAL}.csv')